In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import cvxpy as cp
from math import ceil

data = pd.read_excel('data.xlsx', header=None).values.flatten()

In [2]:
requirements = np.concatenate([data, [102., 102., 103., 103., 104., 104., 105., 105.]])
requirements

array([ 11.,   5.,   4.,   7.,  16.,   6.,   5.,   7.,  13.,   6.,   5.,
         7.,  12.,   5.,   4.,   6.,   9.,   5.,   5.,  11.,  29.,  21.,
        17.,  20.,  27.,  13.,   9.,  10.,  16.,   6.,   5.,   7.,  11.,
         5.,   5.,   6.,  12.,   7.,   7.,  10.,  15.,  10.,   9.,  11.,
        15.,  10.,  10.,  16.,  26.,  21.,  23.,  36.,  50.,  45.,  45.,
        49.,  57.,  43.,  40.,  44.,  52.,  43.,  42.,  45.,  52.,  41.,
        39.,  41.,  48.,  35.,  34.,  35.,  42.,  34.,  36.,  43.,  55.,
        48.,  54.,  65.,  80.,  70.,  74.,  85., 101.,  89.,  88.,  90.,
       100.,  87.,  88.,  89., 104.,  89.,  89.,  90., 106.,  96.,  94.,
        99., 109.,  99.,  96., 102., 102., 102., 103., 103., 104., 104.,
       105., 105.])

## 方法：先进行非整数规划，再进行整数规划

In [3]:
total_week = 112
requirements = requirements[0:total_week]
K = 20
waste_rate = 0.1

In [4]:
working_hands = cp.Parameter(total_week)
waste_hands = cp.Parameter(total_week)
working_hands.value = 4 * requirements
waste_hands.value = np.array([
    4 * round(x * waste_rate) if x % 1 != 0.5 else ceil(x * waste_rate) 
    for x in requirements
])

teaching_hands = cp.Variable(total_week, integer=True)
resting_hands = cp.Variable(total_week, integer=True)
new_hands = cp.Variable((3, total_week), integer=True)

# 机械手购买的分段点
hand_price = cp.Parameter((3, 1))
hand_price.value = np.array([100, 90, 80]).reshape(3, 1)

hand_y = cp.Variable((3, total_week), boolean=True)

In [5]:
working_hands.value, waste_hands.value

(array([ 44.,  20.,  16.,  28.,  64.,  24.,  20.,  28.,  52.,  24.,  20.,
         28.,  48.,  20.,  16.,  24.,  36.,  20.,  20.,  44., 116.,  84.,
         68.,  80., 108.,  52.,  36.,  40.,  64.,  24.,  20.,  28.,  44.,
         20.,  20.,  24.,  48.,  28.,  28.,  40.,  60.,  40.,  36.,  44.,
         60.,  40.,  40.,  64., 104.,  84.,  92., 144., 200., 180., 180.,
        196., 228., 172., 160., 176., 208., 172., 168., 180., 208., 164.,
        156., 164., 192., 140., 136., 140., 168., 136., 144., 172., 220.,
        192., 216., 260., 320., 280., 296., 340., 404., 356., 352., 360.,
        400., 348., 352., 356., 416., 356., 356., 360., 424., 384., 376.,
        396., 436., 396., 384., 408., 408., 408., 412., 412., 416., 416.,
        420., 420.]),
 array([ 4,  0,  0,  4,  8,  4,  0,  4,  4,  4,  0,  4,  4,  0,  0,  4,  4,
         0,  0,  4, 12,  8,  8,  8, 12,  4,  4,  4,  8,  4,  0,  4,  4,  0,
         0,  4,  4,  4,  4,  4,  8,  4,  4,  4,  8,  4,  4,  8, 12,  8,  8,
        16

In [6]:
cons = [
    resting_hands[0] + teaching_hands[0] + working_hands[0] == 50,
    K * teaching_hands >= cp.sum(new_hands, axis=0),

    teaching_hands >= 0,
    resting_hands >= 0,
    new_hands >= 0,

    20 * hand_y[1] <= new_hands[0],
    new_hands[0] <= 20 * hand_y[0],
    20 * hand_y[2] <= new_hands[1],
    new_hands[1]  <= 20 * hand_y[1],
    new_hands[2] <= 10000 * hand_y[2]
]

In [7]:
for t in range(total_week - 1):
    cons.append(resting_hands[t + 1] >= working_hands[t] - waste_hands[t]),
    cons.append(resting_hands[t + 1] + working_hands[t + 1] + teaching_hands[t + 1] == \
        resting_hands[t] + working_hands[t] + teaching_hands[t] + cp.sum(new_hands, axis=0)[t] - waste_hands[t]
        )

In [8]:
obj = cp.Minimize(
    cp.sum(cp.multiply(new_hands, hand_price))\
    + 5 * cp.sum(resting_hands)\
    + 10 * cp.sum(teaching_hands)\
    + 10 * cp.sum(new_hands)
)

In [9]:
prob = cp.Problem(obj, cons)
prob.solve(solver=cp.CPLEX, verbose=True)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 04 12:07:28 AM: Your problem has 896 variables, 232 constraints, and 227 parameters.
(CVXPY) May 04 12:07:28 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 04 12:07:28 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 04 12:07:28 AM: Compiling problem (target solver=CPLEX).
(CVXPY) May 04 12:07:28 AM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> CPLEX
(CVXPY) May 04 12:07:28 AM: Applying reduction CvxAttr2Constr
(CVXPY) May 04 12:07:28 AM: Applying reduction Qp2Symb

Tried aggregator 3 times.
MIP Presolve eliminated 785 rows and 115 columns.
MIP Presolve modified 4 coefficients.
Aggregator did 78 substitutions.
Reduced MIP has 592 rows, 703 columns, and 2073 nonzeros.
Reduced MIP has 222 binaries, 481 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.64 ticks)
Probing fixed 0 vars, tightened 93 bounds.
Probing time = 0.00 sec. (0.56 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 4 coefficients.
Reduced MIP has 592 rows, 703 columns, and 2073 nonzeros.
Reduced MIP has 222 binaries, 481 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.23 ticks)
Probing fixed 0 vars, tightened 7 bounds.
Probing time = 0.00 sec. (0.55 ticks)
Clique table members: 111.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 20 threads.
Root relaxation solution time = 0.00 sec. (2.13 ticks)

        Nodes                                    

357530.0000000042

In [10]:
working_bodies = cp.Parameter(total_week)
waste_bodies = cp.Parameter(total_week)
working_bodies.value = requirements
waste_bodies.value = np.array([
    round(x * waste_rate) if x % 1 != 0.5 else ceil(x * waste_rate) 
    for x in requirements
])

resting_bodies = cp.Variable(total_week, integer=True)
new_bodies = cp.Variable((3, total_week), integer=True)

# 机械手购买的分段点
body_price = cp.Parameter((3, 1))
body_price.value = np.array([200, 180, 160]).reshape(3, 1)

body_y = cp.Variable((3, total_week), boolean=True)

In [11]:
cons2 = [
    resting_bodies[0] + working_bodies[0] == 13,

    resting_bodies >= 0,
    new_bodies >= 0,

    5 * body_y[1] <= new_bodies[0],
    new_bodies[0] <= 5 * body_y[0],
    5 * body_y[2] <= new_bodies[1],
    new_bodies[1]  <= 5 * body_y[1],
    new_bodies[2] <= 10000 * body_y[2]
]

for t in range(total_week - 1):
    cons2.append(resting_bodies[t + 1] + working_bodies[t + 1]== \
        resting_bodies[t] + working_bodies[t] + cp.sum(new_bodies, axis=0)[t] - waste_bodies[t]
        )

In [12]:
obj2 = cp.Minimize(
    cp.sum(cp.multiply(new_bodies, body_price))\
    + 10 * cp.sum(resting_bodies)
)

In [13]:
prob2 = cp.Problem(obj2, cons2)
prob2.solve(solver=cp.CPLEX, verbose=True)

                                     CVXPY                                     
                                     v1.2.0                                    
(CVXPY) May 04 12:08:02 AM: Your problem has 784 variables, 119 constraints, and 227 parameters.
(CVXPY) May 04 12:08:02 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 04 12:08:02 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 04 12:08:02 AM: Compiling problem (target solver=CPLEX).
(CVXPY) May 04 12:08:02 AM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> CPLEX
(CVXPY) May 04 12:08:02 AM: Applying reduction CvxAttr2Constr
(CVXPY) May 04 12:08:02 AM: Applying reduction Qp2Symb

Tried aggregator 2 times.
MIP Presolve eliminated 563 rows and 116 columns.
MIP Presolve added 1 rows and 0 columns.
Aggregator did 111 substitutions.
Reduced MIP has 447 rows, 557 columns, and 1489 nonzeros.
Reduced MIP has 222 binaries, 335 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.75 ticks)
Found incumbent of value 7.9814693e+08 after 0.02 sec. (3.24 ticks)
Probing fixed 0 vars, tightened 2 bounds.
Probing time = 0.00 sec. (1.53 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve modified 3 coefficients.
Reduced MIP has 447 rows, 557 columns, and 1489 nonzeros.
Reduced MIP has 222 binaries, 335 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.02 ticks)
Probing fixed 0 vars, tightened 1 bounds.
Probing time = 0.00 sec. (1.80 ticks)
Clique table members: 111.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 20 threads.
Root relaxation solution time = 0.

104729.99999999948

In [14]:
working_hands.value, resting_hands.value, waste_hands.value

(array([ 44.,  20.,  16.,  28.,  64.,  24.,  20.,  28.,  52.,  24.,  20.,
         28.,  48.,  20.,  16.,  24.,  36.,  20.,  20.,  44., 116.,  84.,
         68.,  80., 108.,  52.,  36.,  40.,  64.,  24.,  20.,  28.,  44.,
         20.,  20.,  24.,  48.,  28.,  28.,  40.,  60.,  40.,  36.,  44.,
         60.,  40.,  40.,  64., 104.,  84.,  92., 144., 200., 180., 180.,
        196., 228., 172., 160., 176., 208., 172., 168., 180., 208., 164.,
        156., 164., 192., 140., 136., 140., 168., 136., 144., 172., 220.,
        192., 216., 260., 320., 280., 296., 340., 404., 356., 352., 360.,
        400., 348., 352., 356., 416., 356., 356., 360., 424., 384., 376.,
        396., 436., 396., 384., 408., 408., 408., 412., 412., 416., 416.,
        420., 420.]),
 array([  5.,  40.,  44.,  30.,  24.,  56.,  56.,  47.,  24.,  48.,  48.,
         39.,  24.,  48.,  52.,  44.,  28.,  40.,  39.,  20.,  91., 111.,
        119.,  98.,  72., 116., 128., 120.,  92., 124., 124., 116.,  96.,
        116., 11

In [15]:
hand_y.value

array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+0

In [16]:
new_hands.value, teaching_hands.value

(array([[ 1.40000000e+01,  0.00000000e+00,  0.00000000e+00,
          2.00000000e+01,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  4.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  8.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00, -8.33327570e-12,  0.00000000e+00,
          1.10000000e+01,  2.00000000e+01,  0.00000000e+00,
         -7.96622846e-12,  0.00000000e+00,  9.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  4.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  2.00000000e+01,  2.00000000e+01,
          0.00000000e+00,  0.00000000e+0

## 输出变量

In [17]:
prob.value + prob2.value

462260.00000000367

In [26]:
answers = [
    np.round(np.sum(new_bodies.value, axis=0)),
    np.round(np.sum(new_hands.value, axis=0)),
    np.round(resting_hands.value),
    np.round(resting_bodies.value),
    np.round(np.sum(new_hands.value, axis=0) + teaching_hands.value),
    np.sum(np.round(new_bodies.value) * body_price.value, axis=0) + \
        np.sum(np.round(new_hands.value) * hand_price.value, axis=0) + \
        5 * np.round(resting_hands.value) + \
        10 * np.round(resting_bodies.value) + \
        10 * np.round(np.sum(new_hands.value, axis=0) + teaching_hands.value)
]

In [27]:
answers[5].sum()

462260.0

In [30]:
answers[5][104:112].sum()

52475.0

In [31]:
answers = np.array(answers).T

In [25]:
answers[25]

array([  0.,   0., 116.,  11.,   0., 690.])

In [32]:
part_ans = np.array(answers[np.arange(104, 112), :], dtype=np.int32)
part_ans = np.concatenate([part_ans, np.sum(part_ans, keepdims=True, axis=0)])

In [33]:
df = pd.DataFrame(part_ans, index=list(range(105, 113)) + ['total'],
    # columns=['购买的容器艇数量', '购买的操作手数量', '保养的操作手数量', '保养的容器艇数量', '参与训练的操作手数量', '总成本']
)
df.to_excel('prob5_part2_answer.xlsx')
df

,0,1,2,3,4,5
105,0,0,413,10,0,2165
106,21,88,368,0,93,14070
107,0,0,417,10,0,2185
108,21,88,372,0,93,14090
109,0,0,421,10,0,2205
110,21,83,376,0,88,13660
111,0,0,420,10,0,2200
112,0,0,380,0,0,1900
total,63,259,3167,40,274,52475


## 问题五·方案一开头条件